In [1]:
import os

In [2]:
%pwd

'd:\\vehicle_insurance_fraud_detection\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\vehicle_insurance_fraud_detection'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/ragalahariakula/vehicle_insurance_fraud_detection.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="ragalahariakula"
os.environ["MLFLOW_TRACKING_PASSWORD"]="33d343356cb0b05950336782d39ca02d51a6f551"

In [17]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    X_test_data_path: Path
    y_test_data_path: Path
    model_path: Path
    metric_file_name: Path
    mlflow_uri: str

In [18]:
from src.vifd.constants import *
from src.vifd.utils.common import read_yaml, create_directories

In [19]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=CONFIG_FILE_PATH,
            schema_filepath=SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            X_test_data_path=config.X_test_data_path,
            y_test_data_path=config.y_test_data_path,
            model_path=config.model_path,
            metric_file_name=config.metric_file_name,
            mlflow_uri="https://dagshub.com/ragalahariakula/vehicle_insurance_fraud_detection.mlflow"

        )

        return model_evaluation_config

In [20]:
import os
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np 
import joblib
from src.vifd.utils.common import save_json

In [21]:
class ModelEvaluation:
    def __init__(self, config:ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        f1 = f1_score(actual, pred)
        acc = accuracy_score(actual, pred)
        return f1, acc

    def log_into_mlflow(self):
        X_test = pd.read_csv(self.config.X_test_data_path)
        y_test = pd.read_csv(self.config.y_test_data_path)['fraud_reported_Y'].values
        model = joblib.load(self.config.model_path)

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities = model.predict(X_test)

            f1, acc = self.eval_metrics(y_test, predicted_qualities)
            
            scores = {"f1": f1, "acc": acc}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_metric("f1", f1)
            mlflow.log_metric("acc", acc)

            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="LogisticRegression")
            else:
                mlflow.sklearn.log_model(model, "model")


In [22]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()

except Exception as e:
    raise e

[2024-01-02 00:18:40,990: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-02 00:18:40,994: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-02 00:18:40,997: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-01-02 00:18:40,999: INFO: common: created directory at: artifacts]
[2024-01-02 00:18:41,000: INFO: common: created directory at: artifacts/model_evaluation]
[2024-01-02 00:18:42,026: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


c:\Users\Lahari\PythonIDLE\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'LogisticRegression' already exists. Creating a new version of this model...
2024/01/02 00:18:52 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: LogisticRegression, version 2
Created version '2' of model 'LogisticRegression'.
